In [9]:
from basic_fcn import *
import time
from torch.utils.data import DataLoader
import torch
import gc
import voc
import torchvision.transforms as standard_transforms
import util
import numpy as np
import sys
import math
import copy

class MaskToTensor(object):
    def __call__(self, img):
        return torch.from_numpy(np.array(img, dtype=np.int32)).long()


def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform_(m.weight.data)
        torch.nn.init.normal_(m.bias.data) #xavier not applicable for biases

In [10]:
mean_std = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
input_transform = standard_transforms.Compose([
        standard_transforms.ToTensor(),
        standard_transforms.Normalize(*mean_std)
    ])
target_transform = MaskToTensor()

train_dataset =voc.VOC('train', transform=input_transform, target_transform=target_transform)
val_dataset = voc.VOC('val', transform=input_transform, target_transform=target_transform)
test_dataset = voc.VOC('test', transform=input_transform, target_transform=target_transform)

train_loader = DataLoader(dataset=train_dataset, batch_size= 16, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size= 16, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size= 16, shuffle=False)

epochs = 20

n_class = 21

fcn_model = FCN(n_class=n_class)
fcn_model.apply(init_weights)

FCN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bnd1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bnd2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bnd3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bnd4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bnd5): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (deconv1): ConvTranspose2d(512, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
  (bn1): Ba

In [11]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

optimizer = torch.optim.Adam(fcn_model.parameters(), lr=1e-2)

# Choose an appropriate loss function from https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html
criterion = nn.CrossEntropyLoss()
fcn_model = fcn_model.to(device)
earlystop = 3
max_model = fcn_model

In [16]:
def train():
    """
    Train a deep learning model using mini-batches.

    - Perform forward propagation in each epoch.
    - Compute loss and conduct backpropagation.
    - Update model weights.
    - Evaluate model on validation set for mIoU score.
    - Save model state if mIoU score improves.
    - Implement early stopping if necessary.

    Returns:
        None.
    """

    patience = 0
    best_iou_score = 0.0
    losses = []

    for epoch in range(epochs):
        ts = time.time()
        for iter, (inputs, labels) in enumerate(train_loader):
            # TODO / DONE  reset optimizer gradients
            
            optimizer.zero_grad()

            # both inputs and labels have to reside in the same device as the model's
            inputs =  inputs.to(device)# TODO / DONE transfer the input to the same device as the model's
            labels =  labels.to(device) # TODO / DONE transfer the labels to the same device as the model's

            outputs = fcn_model(inputs) # TODO / DONE Compute outputs. we will not need to transfer the output, it will be automatically in the same device as the model's!

            loss = criterion(outputs, labels) #TODO / DONE calculate loss

            loss.backward() # TODO / DONE backpropagate

            optimizer.step() # TODO / DONE update the weights
            
            losses.append(loss.item())


            #if iter % 10 == 0:
            #    print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss.item()))

        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        print("Train Avg Loss: {}".format(np.mean(losses)))

        current_miou_score = val(epoch)

        if current_miou_score > best_iou_score:
            best_iou_score = current_miou_score
            patience = 0
            max_model = copy.deepcopy(fcn_model)
            # save the best model
        else:
            patience += 1
            
        # Early stop if patience level is met
        if patience >= earlystop:
            print("Early stop at epoch " + str(epoch))
            break
    

In [20]:
def val(epoch):
    """
    Validate the deep learning model on a validation dataset.

    - Set model to evaluation mode. DONE
    - Disable gradient calculations. DONE
    - Iterate over validation data loader:
        - Perform forward pass to get outputs.
        - Compute loss and accumulate it.
        - Calculate and accumulate mean Intersection over Union (IoU) scores and pixel accuracy.
    - Print average loss, IoU, and pixel accuracy for the epoch.
    - Switch model back to training mode.

    Args:
        epoch (int): The current epoch number.

    Returns:
        tuple: Mean IoU score and mean loss for this validation epoch.
    """
    fcn_model.eval() # Put in eval mode (disables batchnorm/dropout) !
    
    losses = []
    mean_iou_scores = []
    accuracy = []
    
    with torch.no_grad(): # we don't need to calculate the gradient in the validation/testing

        # Iterate through Validation Set
        for iter, (input, label) in enumerate(val_loader):
            # label = (16, 224, 224) / batch size 16 of 244*244 masks
            # output = (16, 21, 224, 224) / batch size 16 of 21 possible classes of 244*244 masks
            
            input =  input.to(device)
            label = label.to(device)
    
            # Perform forward pass to get outputs.
            output = fcn_model.forward(input)
            N, numClass, H, W = output.shape

            # Find the prediction for each pixel
            prediction = output.view(N, n_class, -1).argmax(dim=1).view(N, H, W)
            
            # Compute loss and accumulate it.
            loss = criterion(output, label)
            losses.append(loss.item())
            
            # Calculate and accumulate mean Intersection over Union (IoU) scores and pixel accuracy.
            meanIOU = util.iou(prediction, label, n_class)
            mean_iou_scores.append(meanIOU)

            acc = util.pixel_acc(prediction, label)
            accuracy.append(acc)
    
    print(f"Validation Loss: {np.mean(losses)}")
    print(f"Validation IoU: {np.mean(mean_iou_scores)}")
    print(f"Validation Pixel Acc: {np.mean(accuracy)}")
    print("\n")

    fcn_model.train() #TURNING THE TRAIN MODE BACK ON TO ENABLE BATCHNORM/DROPOUT!!

    return np.mean(mean_iou_scores)

#  #TODO
def modelTest():
    """
    Test the deep learning model using a test dataset.

    - Load the model with the best weights.
    - Set the model to evaluation mode.
    - Iterate over the test data loader:
        - Perform forward pass and compute loss.
        - Accumulate loss, IoU scores, and pixel accuracy.
    - Print average loss, IoU, and pixel accuracy for the test data.
    - Switch model back to training mode.

    Returns:
        None. Outputs average test metrics to the console.
    """

    # Asssume model loaded with the best weights.
    fcn_model = max_model
    
    fcn_model.eval()  # Put in eval mode (disables batchnorm/dropout) !

    losses = []
    mean_iou_scores = []
    accuracy = []

    with torch.no_grad():  # we don't need to calculate the gradient in the validation/testing

        # Iterate through Test Set
        for iter, (input, label) in enumerate(test_loader):

            input =  input.to(device)
            label = label.to(device)

            # Perform forward pass to get outputs.
            output = fcn_model.forward(input)
            N, numClass, H, W = output.shape

            # Find the prediction for each pixel
            prediction = output.view(N, n_class, -1).argmax(dim=1).view(N, H, W)
            
            # Compute loss and accumulate it.
            loss = criterion(output, label)
            losses.append(loss.item())
            
            # Calculate and accumulate mean Intersection over Union (IoU) scores and pixel accuracy.
            meanIOU = util.iou(prediction, label, n_class)
            mean_iou_scores.append(meanIOU)

            acc = util.pixel_acc(prediction, label)
            accuracy.append(acc)

    print(f"Test Loss at Test: {np.mean(losses)}")
    print(f"Test IoU at Test: {np.mean(mean_iou_scores)}")
    print(f"Test Pixel acc at Test: {np.mean(accuracy)}")

    fcn_model.train()  #TURNING THE TRAIN MODE BACK ON TO ENABLE BATCHNORM/DROPOUT!!

In [18]:
def exportModel(inputs):    
    """
    Export the output of the model for given inputs.

    - Set the model to evaluation mode.
    - Load the model with the best saved weights.
    - Perform a forward pass with the model to get output.
    - Switch model back to training mode.

    Args:
        inputs: Input data to the model.

    Returns:
        Output from the model for the given inputs.
    """

    fcn_model.eval() # Put in eval mode (disables batchnorm/dropout) !
    
    saved_model_path = "Fill Path To Best Model"
    # TODO Then Load your best model using saved_model_path
    
    inputs = inputs.to(device)
    
    output_image = fcn_model(inputs)
    
    fcn_model.train()  #TURNING THE TRAIN MODE BACK ON TO ENABLE BATCHNORM/DROPOUT!!
    
    return output_image

# if __name__ == "__main__":

#     val(0)  # show the accuracy before training
#     train()
#     modelTest()

#     # housekeeping
#     gc.collect()
#     torch.cuda.empty_cache()

In [19]:
train()

Finish epoch 0, time elapsed 10.932702541351318
Train Avg Loss: 2.2333646757262096
Validation Loss: 1.4381170315401894
Validation IoU: 0.053691666031312806
Validation Pixel Acc: 0.7392254264987247


Finish epoch 1, time elapsed 10.668454885482788
Train Avg Loss: 1.7796936631202698
Validation Loss: 1.3433957993984222
Validation IoU: 0.05372888506981775
Validation Pixel Acc: 0.7397263062591107


Finish epoch 2, time elapsed 10.195504665374756
Train Avg Loss: 1.5972679158051808
Validation Loss: 1.4163699405533927
Validation IoU: 0.05564781345682834
Validation Pixel Acc: 0.7508232094456085


Finish epoch 3, time elapsed 10.672451972961426
Train Avg Loss: 1.5055835034166063
Validation Loss: 1.2982743297304427
Validation IoU: 0.05564781345682834
Validation Pixel Acc: 0.7508232094456085


Finish epoch 4, time elapsed 10.23966932296753
Train Avg Loss: 1.4469209160123553
Validation Loss: 1.2275715725762504
Validation IoU: 0.05564781345682834
Validation Pixel Acc: 0.7508232094456085


Finish epo

In [21]:
modelTest()

Test Loss at Test: 1.4566880294254847
Test IoU at Test: 0.05575184499509766
Test Pixel acc at Test: 0.7289345076758381


## AFTER COSINE ANNEALING LR

In [76]:
fcn_model = FCN(n_class=n_class)
fcn_model.apply(init_weights)
fcn_model = fcn_model.to(device)

earlystop = 3
max_model = fcn_model

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fcn_model.parameters(), lr=1e-2)

#Test Cosine Annealing Learning Rate
iterMax = math.floor(len(train_dataset)/16)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, iterMax, 0)

In [77]:
def train1():
    patience = 0
    best_iou_score = 0.0
    losses = []

    for epoch in range(epochs):
        ts = time.time()
        for iter, (inputs, labels) in enumerate(train_loader):
            
            optimizer.zero_grad()

            inputs, labels = inputs.to(device), labels.to(device)

            outputs = fcn_model(inputs) 

            loss = criterion(outputs, labels)

            loss.backward() 

            optimizer.step()
            
            losses.append(loss.item())
            
            scheduler.step() # For cosine annealing learning rate

        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        print("Train Avg Loss: {}".format(np.mean(losses)))
        
        current_miou_score = val(epoch)

        if current_miou_score > best_iou_score:
            best_iou_score = current_miou_score
            patience = 0
            max_model = copy.deepcopy(fcn_model)
            # save the best model
        else:
            patience += 1
            
        # Early stop if patience level is met
        if patience >= earlystop:
            print("Early stop at epoch " + str(epoch))
            break
    

In [30]:
train1()

KeyboardInterrupt: 

In [25]:
modelTest()

Test Loss at Test: 1.3813342452049255
Test IoU at Test: 0.05656007269638597
Test Pixel acc at Test: 0.7286482830436861


## Weight Imbalance

In [101]:
#TODO Get class weights
def getClassWeights():
    ans = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

    for iter, (inputs, labels) in enumerate(train_loader):  
        unique_elements, counts = torch.unique(labels, return_counts=True)

        for i in range(len(unique_elements)):
            ans[unique_elements[i]] += counts[i]

    normalized = [tensor.tolist() for tensor in ans]
    #normalized = [num/sum(normalized) for num in normalized]
    normalized = [1/(1-pow(0.1,num/100000)) for num in normalized]
    #normalized[0] = 0.01
    return torch.tensor(normalized)

In [102]:
class_weights = getClassWeights()
print(class_weights)
class_weights = class_weights.to(device)

tensor([1.0000, 1.4908, 1.5823, 1.1482, 1.5141, 1.2422, 1.0103, 1.2189, 1.0136,
        1.0838, 1.1975, 1.0506, 1.0377, 1.1603, 1.1925, 1.0000, 1.9344, 2.4230,
        1.0209, 1.0675, 1.0520])


In [103]:
fcn_model = FCN(n_class=n_class)
fcn_model.apply(init_weights)
fcn_model = fcn_model.to(device)

earlystop = 3
max_model = fcn_model

optimizer = torch.optim.Adam(fcn_model.parameters(), lr=1e-2)
criterion = nn.CrossEntropyLoss(class_weights)

In [104]:
train1()

Finish epoch 0, time elapsed 10.47522783279419
Train Avg Loss: 3.5078877891813005
Validation Loss: 3.1562504938670566
Validation IoU: 0.005710405006639658
Validation Pixel Acc: 0.019417703255967565


Finish epoch 1, time elapsed 10.486026048660278
Train Avg Loss: 2.9330755174160004
Validation Loss: 2.643634021282196
Validation IoU: 0.04564813260259882
Validation Pixel Acc: 0.5360647485138029


Finish epoch 2, time elapsed 10.371671199798584
Train Avg Loss: 2.4552833239237466


KeyboardInterrupt: 

In [82]:
modelTest()

Test Loss at Test: 2.2305848683629717
Test IoU at Test: 0.05575184499509766
Test Pixel acc at Test: 0.7289345076758381


In [ ]:
# weights (num /100000)
# Test Loss at Test: 1.4747441240719386
# Test IoU at Test: 0.05621371719732892
# Test Pixel acc at Test: 0.729007821041363

In [ ]:
# BASELINE SCORES
# Test Loss at Test: 1.4566880294254847
# Test IoU at Test: 0.05575184499509766
# Test Pixel acc at Test: 0.7289345076758381